In [1]:
import os
from pandas import read_csv, read_excel, DataFrame
import sae
import gurobipy as gp
from gurobipy import GRB
import numpy as np

import sys
sys.path.append('../../')
import saedfsc

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
numCustomersTypes = 3
numSubsystems = 10
numObjectives = 11

# Create a list of supplier names
names = ['CustomerType' + str(i) for i in range(1, numCustomersTypes + 1)]

qty = np.random.randint(low=25, high=1000, size=numCustomersTypes)
meanDemand = np.random.randint(low=15, high=30, size=numCustomersTypes)
stdDevDemand = np.random.randint(low=1, high=5, size=numCustomersTypes)
priceFocus = np.random.random(size=numCustomersTypes)

rawPartworthWts = [np.random.random(numSubsystems) for _ in range(numCustomersTypes)]
partworthWts = [np.array2string(rawPartworthWts[c]/sum(rawPartworthWts[c]), separator=',') for c in range(numCustomersTypes)]
rawPerfWts = [np.random.random(numObjectives) for _ in range(numCustomersTypes)]
perfWts = [np.array2string(rawPerfWts[c]/sum(rawPerfWts[c]), separator=',') for c in range(numCustomersTypes)]

# Create the DataFrame
df = DataFrame({
    'Name': names,
    'Quantity': qty,
    'Mean Demand': meanDemand,
    'Std Dev Demand': stdDevDemand,
    'PriceFocus': priceFocus,
    'PartworthUtilityWeights': partworthWts,
    'PerformanceUtilityWeights': perfWts
})
df.to_csv('../resources/customers-{}.csv'.format(numCustomersTypes), index=False)
customers: DataFrame = read_csv('../resources/customers-{}.csv'.format(numCustomersTypes))
customers

,Name,Quantity,Mean Demand,Std Dev Demand,PriceFocus,PartworthUtilityWeights,PerformanceUtilityWeights
0,CustomerType1,550,23,4,0.185002,"[0.05502635,0.15548226,0.1138455 ,0.15706312,0...","[0.10403705,0.17744796,0.11303432,0.01412367,0..."
1,CustomerType2,32,19,2,0.721966,"[0.05439101,0.0698292 ,0.1680139 ,0.07585734,0...","[0.19323785,0.15236526,0.06466306,0.07034756,0..."
2,CustomerType3,804,21,4,0.489219,"[0.09282521,0.08616018,0.1542277 ,0.13957106,0...","[0.13414659,0.13715431,0.03786418,0.11151464,0..."


Print dictionary of customers and their weights.

In [3]:
name_weights_dict = customers.set_index('Name')['PerformanceUtilityWeights'].to_dict()
cWts = {c : np.fromstring(name_weights_dict[c].strip('[]'), sep=',') for c in name_weights_dict}
cWts

{'CustomerType1': array([0.10403705, 0.17744796, 0.11303432, 0.01412367, 0.09074952,
        0.06196034, 0.04152583, 0.07723446, 0.08962796, 0.14334552,
        0.08691335]),
 'CustomerType2': array([0.19323785, 0.15236526, 0.06466306, 0.07034756, 0.00089544,
        0.00064008, 0.12437685, 0.1397965 , 0.07107653, 0.14266048,
        0.03994039]),
 'CustomerType3': array([0.13414659, 0.13715431, 0.03786418, 0.11151464, 0.12350755,
        0.12949936, 0.13148834, 0.01641187, 0.05542951, 0.06425334,
        0.05873032])}

In [4]:
cQty = dict(zip(customers['Name'], customers['Quantity'])) # customer quantities
cQty

{'CustomerType1': 550, 'CustomerType2': 32, 'CustomerType3': 804}